In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the Data

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gc

In [ ]:
PROJECT_PATH = '/content/drive/My Drive/Datasets/ML Project'
MODEL_NAME = 'linear_regression'
WITH_PREPROCESSING = True
FIGURES_PATH = f'{PROJECT_PATH}/figures/{MODEL_NAME}/{"with" if WITH_PREPROCESSING else "without"}'
MODEL_PATH = f'{PROJECT_PATH}/models/{"with" if WITH_PREPROCESSING else "without"}'
def get_figure_path(name: str):
  return f'{FIGURES_PATH}/{name}.png'

DATASETS_PATH = f'{PROJECT_PATH}/data/recommended'
TRAINING_PATH = f'{DATASETS_PATH}/training/training.csv'
TESTING_PATH = f'{DATASETS_PATH}/test/test.csv'
INDEX_COL = 'pkSeqID'
COLUMNS_ORDERED = [
    'min', 'max', 'mean', 'stddev',
    'saddr', 'sport', 'daddr', 'dport',
    'srate', 'drate',
    'N_IN_Conn_P_SrcIP', 'N_IN_Conn_P_DstIP',
    'state_number', 'proto',
    'seq',
    'attack', 'category', 'subcategory'
]

In [ ]:
import joblib

def save_model(model, name = MODEL_NAME):
    joblib.dump(model, f'{MODEL_PATH}/{name}.joblib')

In [ ]:
training = pd.read_csv(TRAINING_PATH, index_col=INDEX_COL)[COLUMNS_ORDERED]
training.head()

,min,max,mean,stddev,saddr,sport,daddr,dport,srate,drate,N_IN_Conn_P_SrcIP,N_IN_Conn_P_DstIP,state_number,proto,seq,attack,category,subcategory
pkSeqID,,,,,,,,,,,,,,,,,,
3142762,0.000000,4.031619,2.687519,1.900363,192.168.100.150,6551,192.168.100.3,80,0.494549,0.0,100,100,4,udp,251984,1,DDoS,UDP
2432264,3.856930,4.012924,3.934927,0.078003,192.168.100.150,5532,192.168.100.3,80,0.256493,0.0,38,100,3,tcp,256724,1,DDoS,TCP
1976315,2.974100,3.609205,3.341429,0.268666,192.168.100.147,27165,192.168.100.3,80,0.294880,0.0,100,100,3,tcp,62921,1,DDoS,TCP
1240757,0.000000,4.942302,3.222832,1.823185,192.168.100.150,48719,192.168.100.3,80,0.461435,0.0,63,63,4,udp,99168,1,DoS,UDP
3257991,2.979995,4.994452,3.983222,0.822418,192.168.100.147,22461,192.168.100.3,80,1.002999,0.0,100,100,4,udp,105063,1,DDoS,UDP


In [ ]:
testing = pd.read_csv(TESTING_PATH, index_col=INDEX_COL)[COLUMNS_ORDERED]

# Final Pipeline

After trying out a bunch of techniques for preprocessing, the scores only worsened.

So a simple pipeline is all that is needed for this particular dataset.

## Handling Target

In [ ]:
def process_categories(cat):
  if str(cat).lower().startswith('theft'):
    return 'Theft'
  if str(cat).lower().startswith('normal'):
    return 'Normal'
  if cat == 'DoS HTTP':
    return 'DoS TCP'
  if cat == 'DDoS HTTP':
    return 'DDoS TCP'
  return cat


In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

cat_encoder_hot = OneHotEncoder()
cat_encoder_label = LabelEncoder()

In [ ]:
X_train = training.drop(['attack' ,'category', 'subcategory'], axis=1)
y_train_cat = pd.DataFrame({'category': (training['category'] + ' ' + training['subcategory']).map(process_categories)})
y_train_hot = cat_encoder_hot.fit_transform(y_train_cat)
y_train_label = cat_encoder_label.fit_transform(y_train_cat['category'])

In [ ]:
X_test = testing.drop(['attack', 'category', 'subcategory'], axis=1)
y_test_cat = pd.DataFrame({'category': (testing['category'] + ' ' + testing['subcategory']).map(process_categories)})
y_test_hot = cat_encoder_hot.transform(y_test_cat)
y_test_label = cat_encoder_label.transform(y_test_cat['category'])

In [ ]:
del training
del testing
gc.collect()

0

## Preprocessing Pipeline

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import cross_validate, train_test_split

In [ ]:
def process_port(p):
  return int(p, 16) if str(p).startswith('0x') else int(p)

def process_ports(ports: pd.DataFrame):
  return ports.map(process_port)


In [ ]:
DEFAULT_RATE_SHIFT = 1.1

In [ ]:
def shift_and_log(data, shift=DEFAULT_RATE_SHIFT): # 1.1 So that the output has no zero values, and a small change is not that significant
  return np.log10(data + shift)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

srate_idx, drate_idx = [list(X_train.columns).index('srate'), list(X_train.columns).index('drate')]
class CombinedFeatureAdder(BaseEstimator, TransformerMixin):
  def __init__(self, normalize=True) -> None:
    super().__init__()
    self.normalize = normalize

  def fit(self, X, y=None):
    return self

  def transform(self, X: pd.DataFrame, y=None):
    srate_to_drate = np.log10(X.loc[:, 'srate'] + DEFAULT_RATE_SHIFT) / np.log10(X.loc[:, 'drate'] + DEFAULT_RATE_SHIFT)
    if self.normalize:
      return X.assign(srate_to_drate=np.log1p(srate_to_drate))
    else:
      return X.assign(srate_to_drate=srate_to_drate)

In [ ]:
TO_DROP = ['saddr', 'daddr', 'seq']

data_cleaner = ColumnTransformer([
  ('drop', 'drop', TO_DROP),
  ('encode', OneHotEncoder(), ['proto']),
  ('port', FunctionTransformer(func=process_ports), ['sport', 'dport']),
  ('rate', FunctionTransformer(func=shift_and_log), ['srate', 'drate']),
], remainder='passthrough')

preprocessing = Pipeline([
    ('augment', CombinedFeatureAdder()),
    ('clean', data_cleaner),
    ('std', StandardScaler()),
])

In [ ]:
%%script true
X_train_cleaned = preprocessing.fit_transform(X_train)